In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

parquet_input_path = '/home/abhay/work/dream11/processed_output/delivery_parquet'

spark = SparkSession.builder.appName('SparkByExamples.com')\
    .config('spark.driver.bindAddress','localhost')\
    .config("spark.ui.port","4051")\
    .config("spark.driver.memory","5g")\
    .getOrCreate()

t20_df_with_boundaries = spark.read.parquet(parquet_input_path)\
    .where(f.col('match_type') == 'T20') \
    .where(f.col('dt') > '2016-12-31') \
    .withColumn('is_boundary', f.when((f.col('batter_runs') >= 4) & (f.col("batter_runs") < 6), 1).otherwise(0)) \
    .withColumn('is_six', f.when(f.col("batter_runs") >= 6, 1).otherwise(0)) \
    .withColumn('is_dismissed', f.when(f.col("wicket_player_id") == f.col("batter_id"), 1).otherwise(0))

23/02/12 12:33:01 WARN Utils: Your hostname, Chaturvedi_PC resolves to a loopback address: 127.0.1.1; using 172.27.64.6 instead (on interface eth0)
23/02/12 12:33:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/12 12:33:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
fielder_relevant_dimensions = ["dt","wicket_fielder_id", "wicket_fielder_name", "bowler_name", "bowler_id"] 
fielder_relevant_metrics = ["is_dismissed"]

In [7]:
from pyspark.sql.functions import col, trunc
t20_batter_fielder_df = t20_df_with_boundaries\
    .select(fielder_relevant_dimensions+fielder_relevant_metrics) \
    .filter(col("wicket_fielder_id").isNotNull()) \
    .withColumn("dt", trunc("dt", "month"))

In [9]:
from pyspark.sql.functions import sum, count, when, col

t20_batter_fielder_df_per_month = t20_batter_fielder_df.groupBy(fielder_relevant_dimensions) \
    .agg(
        sum("is_dismissed").alias("wicket_sum")
    )

In [10]:
t20_batter_fielder_df_per_month.toPandas()

,dt,wicket_fielder_id,wicket_fielder_name,bowler_name,bowler_id,wicket_sum
0,2021-09-01,b0946605,AS Joseph,KOK Williams,78cb83f3,2
1,2022-04-01,b0482a1d,Tilak Varma,DR Sams,caf69bf7,3
2,2019-02-01,52cb36a0,Manisankar Murasingh,Kamal Das,1d09dda7,1
3,2021-11-01,6f7e30d6,JE Dooley,S Molineux,99e23670,3
4,2021-11-01,d1c36f5c,JJ Roy,AU Rashid,249d60c9,1
...,...,...,...,...,...,...
38711,2019-03-01,77255a9e,RA Tripathi,BA Stokes,e087956b,1
38712,2022-09-01,aea89d48,L Keny,KKN Egodage,5bb904d4,1
38713,2017-05-01,0994d0ae,V Shankar,B Kumar,2e81a32d,1
38714,2020-09-01,abb83e27,JM Bairstow,Rashid Khan,5f547c8b,1
